Neural Network for Satellite Data Classification Using Tensorflow

In [ ]:
import os
from pyrsgis import raster

In [2]:
Input_Data = 'D:\DL_WORK\Sentinal_SEPT_Jesus_Maria.tif'
Input_Classified = 'D:\DL_Work\Re_Clsied_APR_Jesus_Maria.tif'
Input_Prediction = 'D:\DL_Work\Sentinel2_Sept_Amacueca.tif'

In [3]:
# Read the rasters as array
ds1, features_Input = raster.read(Input_Data, bands='all')
ds2, label_Classified = raster.read(Input_Classified, bands=1)
ds3, features_Prediction = raster.read(Input_Prediction, bands='all')

In [4]:
print("multispectral image shape: ", features_Input.shape)
print("binary Classified image shape: ", label_Classified.shape)
print("Prediction multispectral image shape: ", features_Prediction.shape)

multispectral image shape:  (4, 2861, 3663)
binary Classified image shape:  (2861, 3663)
Prediction multispectral image shape:  (4, 1789, 1644)


In [5]:
from pyrsgis.convert import changeDimension

features_Input = changeDimension(features_Input)
label_Classified = changeDimension (label_Classified)
features_Prediction = changeDimension(features_Prediction)
nBands = features_Input.shape[1]
label_Input_Feature = (label_Classified == 1).astype(int)

print("Input multispectral image shape: ", features_Input.shape)
print("binary built-up image shape: ", label_Classified.shape)
print("Prediction multispectral image shape: ", features_Prediction.shape)

Input multispectral image shape:  (10479843, 4)
binary built-up image shape:  (10479843,)
Prediction multispectral image shape:  (2941116, 4)


In [6]:
from sklearn.model_selection import train_test_split

xTrain, xTest, yTrain, yTest = train_test_split(features_Input, label_Classified, test_size=0.4, random_state=42)

print(xTrain.shape)
print(yTrain.shape)

print(xTest.shape)
print(yTest.shape)

(6287905, 4)
(6287905,)
(4191938, 4)
(4191938,)


In [7]:
# Normalise the data
xTrain = xTrain / 255.0
xTest = xTest / 255.0
features_Prediction = features_Prediction / 255.0

# Reshape the data
xTrain = xTrain.reshape((xTrain.shape[0], 1, xTrain.shape[1]))
xTest = xTest.reshape((xTest.shape[0], 1, xTest.shape[1]))
features_Prediction = features_Prediction.reshape((features_Prediction.shape[0], 1, features_Prediction.shape[1]))

# Print the shape of reshaped data
print(xTrain.shape, xTest.shape, features_Prediction.shape)

(6287905, 1, 4) (4191938, 1, 4) (2941116, 1, 4)


In [8]:
from tensorflow import keras

# Define the parameters of the model
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(1, nBands)),
    keras.layers.Dense(14, activation='relu'),
    keras.layers.Dense(2, activation='softmax')])

# Define the accuracy metrics and parameters
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Run the model
model.fit(xTrain, yTrain, epochs=2)

Epoch 1/2
196498/196498 [==============================] - 100s 511us/step - loss: 0.2694 - accuracy: 0.8944
Epoch 2/2
196498/196498 [==============================] - 144s 732us/step - loss: 0.2584 - accuracy: 0.8987


In [9]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score

# Predict for test data 
yTestPredicted = model.predict(xTest)
yTestPredicted = yTestPredicted[:,1]

# Calculate and display the error metrics
yTestPredicted = (yTestPredicted>0.5).astype(int)
cMatrix = confusion_matrix(yTest, yTestPredicted)
pScore = precision_score(yTest, yTestPredicted)
rScore = recall_score(yTest, yTestPredicted)

print("Confusion matrix: for 14 nodes\n", cMatrix)
print("\nP-Score: %.3f, R-Score: %.3f" % (pScore, rScore))

Confusion matrix: for 14 nodes
 [[3629915   54399]
 [ 365607  142017]]

P-Score: 0.723, R-Score: 0.280


In [10]:
import numpy as np
predicted = model.predict(features_Prediction)
predicted = predicted[:,1]

#Export raster
prediction = np.reshape(predicted, (ds3.RasterYSize, ds3.RasterXSize))
outFile = 'D:\DL_WORK\ANN_Sept_Amacueca12.tif'
raster.export(prediction, ds3, filename=outFile, dtype='float')